In [15]:
# !conda create --name faiss-tutorial python=3.12
# !activate faiss-tutorial
# !pip install jupyter
!python --version
print('---')
# !pip list

Python 3.12.3
---


In [16]:
# !pip install pandas

In [17]:
# !pip show pandas

In [8]:
import requests
from io import StringIO
import pandas as pd

In [9]:
res = 'SICK_train.txt'
# create dataframe
data = pd.read_csv((res), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [13]:
# pandas.core.frame.DataFrame
type(data)

pandas.core.frame.DataFrame

In [14]:
# pandas.core.series.Series
type(data['sentence_A'])

pandas.core.series.Series

In [ ]:
data['sentence_A']

In [18]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [19]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [20]:
data_file_names = [
    'MSRpar.train.tsv',
    'MSRpar.test.tsv',
    'OnWN_2012.test.tsv',
    'OnWN_2013.test.tsv',
    'OnWN_2014.test.tsv',
    'images_2014.test.tsv',
    'images_2015.test.tsv'
]

In [21]:
# each of these dataset have the same structure, so we loop through each creating our sentences data
for file_name in data_file_names:
    # extract to dataframe
    data = pd.read_csv(file_name, sep='\t', header=None, on_bad_lines='skip')
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

In [22]:
len(set(sentences))

14505

In [23]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [27]:
# !pip install sentence_transformers

In [31]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
# model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('D:/models/bert-base-nli-mean-tokens')

In [ ]:
%%time
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

In [48]:
import faiss

In [49]:
d = sentence_embeddings.shape[1]
d

768

In [50]:
index = faiss.IndexFlatL2(d)

In [51]:
index.is_trained

True

In [52]:
index.add(sentence_embeddings)

In [53]:
index.ntotal

14504

In [54]:
k = 4
xq = model.encode(["Someone sprints with a football"])

In [81]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[2612 5175 8188 1198]]
CPU times: total: 0 ns
Wall time: 5.6 ms


In [78]:
print(D)

[[54.623783 54.853485 57.356243 57.90586 ]]


In [91]:
results = [sentences[i] for i in I.tolist()[0]]
results

In [89]:
type(I.tolist())
print(I.tolist())

[[2612, 5175, 8188, 1198]]


In [31]:
data[1]

0                         Small dog chews on a big stick.
1                       A tennis player hitting the ball.
2        a lone snowboarder in the middle of a snowy gust
3              A pair of dogs playing with a purple ball.
4                              a bird lands in the water.
                              ...                        
1495                 A man doing a trick on a skateboard.
1496    A young girl in swim goggles does the backstro...
1497                                A deer jumps a fence.
1498    A young girl dressed in a Minnie mouse outfit ...
1499                      a dog sprints across the water.
Name: 1, Length: 1500, dtype: object

In [33]:
sentences[:5]

['A black and white dog is chasing a ping Frisbee.',
 'Two men in a running contest.',
 'the selection of a lottery winner',
 'the fashionable elite of a given community',
 'A man in red and black is hitting a soccer ball with his head.']

In [73]:
sentences[4586]

IndexError: list index out of range

In [72]:
data['sentence_A']

0       A group of kids is playing in a yard and an ol...
1       A group of children is playing in the house an...
2       The young boys are playing outdoors and the ma...
3       The kids are playing outdoors near a man with ...
4       The young boys are playing outdoors and the ma...
                              ...                        
4495                      A door is being opened by a man
4496                     Someone is boiling okra in a pot
4497    The man is singing heartily and playing the gu...
4498          A man in blue has a yellow ball in the mitt
4499                 Three dogs are resting on a sidewalk
Name: sentence_A, Length: 4500, dtype: object